# Session 4: Word Vectors

Word vectors (also known as 'word embeddings') are one of the most popular kinds of AI models. They are extremely useful in many domains. In essence, a word vector is a set of numbers that attempt to capture the meaning of a word. In typical implementations, each word is represented by a set of 200-300 numbers. In linear algebra, a one-dimensional array of numbers is known as a 'vector', hence these sets of numbers representing words' meanings are known as 'word vectors'.

Using neural networks, we can expose the computer to a large amount of text, and allow it to learn an appropriate set of numbers for each word it encounters. In this notebook, we will learn about the most famous of all word vector algorithms, `word2vec`, which was first described by Tomas Mikolov and his team in 2013:

* Tomas Mikolov, Ilya Sutskever, and others, ‘Distributed Representations of Words and Phrases and Their Compositionality’, in Advances in Neural Information Processing Systems 26, ed. by C. J. C. Burges and others (Curran Associates, Inc., 2013), pp. 3111–19 <http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf>
* Tomas Mikolov, Kai Chen, and others, ‘Efficient Estimation of Word Representations in Vector Space’, ArXiv:1301.3781 Cs, 2013 <http://arxiv.org/abs/1301.3781>.

In fact, `word2vec` is not a single algorithm, but rather a family of similar algorithms. In this session we will consider just the most famous `word2vec` algorithm, namely the `skip-gram model` trained using `negative sampling`.

## Applications of Word Vectors

Word vectors allow the computer to 'understand' language far more effectively. Rather than seeing each word as simply an arbitrarily different object, a computer using word vectors can analyse each word as a point in 200- or 300-dimenstional space. Words that are similar in meaning will have similar word vectors. And as we will see, the spaces between the word vectors are also significant: the words are arranged in patterns that represent their relationships to one another.

Accordingly, most AI systems that process language now include a word vector layer as part of their architecure. When the system encounters some text (e.g. when you speak to Siri or Alexa), your words are converted into word vectors, *and then* the computer examines what the text says and determines how it should respond.

In the Humanities, word vectors have become a popular modelling tool, because they allow researchers to perform sophisticated analysis on large corpora of text. Some examples include:

* [The Women Writers Vector Toolkit](https://wwp.northeastern.edu/lab/wwvt/index.html)
* William L. Hamilton, Jure Leskovec, and Dan Jurafsky, ‘Diachronic Word Embeddings Reveal Statistical Laws of Semantic Change’, ArXiv:1605.09096 [Cs], 2018 <http://arxiv.org/abs/1605.09096>.
* Ryan Heuser, 'Semantic Networks' <https://ryanheuser.org/word-vectors-4/>

## Training a `word2vec` model in Gensim

It is very easy to train a `word2vec` model in Gensim, which includes Mikolov's original `word2vec` code in its codebase.

In [1]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data
corpus = api.load('text8')
# api.info()

[==================================================] 100.0% 31.6/31.6MB downloaded


text8 is first 100MB from English wikipedia, 17 million words

The file is downloadable from 
http://mattmahoney.net/dc/text8.zip


In [2]:
# Examine the corpus to see what is there
api.info("text8")

{'checksum': '68799af40b6bda07dfa47a32612e5364',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'file_name': 'text8.gz',
 'file_size': 33182058,
 'license': 'not found',
 'num_records': 1701,
 'parts': 1,
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'record_format': 'list of str (tokens)'}

In [10]:
type(corpus)

text8.Dataset

In [11]:
data = [d for d in corpus]

In [6]:
len(data)

1701

In [7]:
len(data[0])

10000

In [8]:
print(data[0][:100])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']


In [9]:
print(data[1][:100])

['reciprocity', 'qualitative', 'impairments', 'in', 'communication', 'as', 'manifested', 'by', 'at', 'least', 'one', 'of', 'the', 'following', 'delay', 'in', 'or', 'total', 'lack', 'of', 'the', 'development', 'of', 'spoken', 'language', 'not', 'accompanied', 'by', 'an', 'attempt', 'to', 'compensate', 'through', 'alternative', 'modes', 'of', 'communication', 'such', 'as', 'gesture', 'or', 'mime', 'in', 'individuals', 'with', 'adequate', 'speech', 'marked', 'impairment', 'in', 'the', 'ability', 'to', 'initiate', 'or', 'sustain', 'a', 'conversation', 'with', 'others', 'stereotyped', 'and', 'repetitive', 'use', 'of', 'language', 'or', 'idiosyncratic', 'language', 'lack', 'of', 'varied', 'spontaneous', 'make', 'believe', 'play', 'or', 'social', 'imitative', 'play', 'appropriate', 'to', 'developmental', 'level', 'restricted', 'repetitive', 'and', 'stereotyped', 'patterns', 'of', 'behavior', 'interests', 'and', 'activities', 'as', 'manifested', 'by', 'at', 'least', 'one']


### Step 1: Set hyperparameters and instantiate model

In [12]:
vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

model = Word2Vec(
    size=vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

### Step 2: Fit model to corpus

In [13]:
# build a model
model.build_vocab(corpus)

In [14]:
# Train the model on the corpus
model.train(sentences=corpus, epochs=5, total_examples=model.corpus_count)

(62532758, 85026035)

### Step 3: Extract word vectors from model

The fully trained model includes all of the weights used to predict the context words for each input word. If you are not planning on training the model further, these weights can be discarded, and you can just keep the weights for the word vectors.

In [15]:
word_vectors = model.wv
del model # Delete the whole model to free up the computer's RAM

### Step 4: Have a play with the model

There are several ways you can use word vectors. One of the most famous is to use them to compute analogies. The formula is:

<center><em>x</em> is to <em>small</em> as <em>biggest</em> is to <em>big</em></center>

$$x - vector('small') = vector('biggest') - vector('big')$$

$$\therefore x = vector('small') + vector('biggest') - vector('big')$$

In [16]:
# See the word vector for a particular word
vector = word_vectors['banana']
print(vector)

[ 0.08057743 -0.37878227  0.11396107  0.14235406 -0.2552257  -0.04763536
 -0.08588961  0.01662323  0.46558335 -0.50837046  0.30185136 -0.06098369
 -0.418174    0.13988963 -0.10220706  0.15339582  0.38421425  0.21753767
 -0.26462123 -0.33933952  0.28586322  0.11846864  0.3968051  -0.16568922
 -0.22783485  0.36126766 -0.22072391  0.16978219  0.5335923   0.18782915
  0.60121    -0.36147678 -0.02602972  0.14496188 -0.17212881 -0.24069406
 -0.29610342 -0.32738495 -0.1662697  -0.38284692  0.01360897  0.06235801
  0.15124774 -0.8239667   0.2916196   0.04945309  0.65051603  0.46133086
 -0.09078005  0.24386574 -0.04115861 -0.1900839  -0.5249951   0.10927019
 -0.2982055   0.14596453 -0.47051612 -0.11326216 -0.14202902 -0.25076422
 -0.24767946 -0.07321762 -0.7660264  -0.3274344   0.18505608  0.18432108
 -0.6316265  -0.0014289   0.10793305 -0.05339458 -0.1794082  -0.15424067
  0.5362082   0.49607205 -0.00949856 -0.24278763  0.28570586  0.6374683
  0.10907999  0.17071956  0.1159452  -0.31039122 -0.

In [21]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('university', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('college', 0.8302913904190063)
('urbana', 0.776872992515564)
('polytechnic', 0.7725688219070435)
('institute', 0.7647371292114258)
('champaign', 0.7477920055389404)
('cambridge', 0.7369121313095093)
('yale', 0.7320483922958374)
('polytechnical', 0.7319600582122803)
('rutgers', 0.7314851880073547)
('johns', 0.7283385396003723)


In [22]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('red', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('yellow', 0.8184937238693237)
('white', 0.8159852623939514)
('blue', 0.8070787191390991)
('green', 0.7698606848716736)
('purple', 0.7313545346260071)
('black', 0.7292963862419128)
('stripes', 0.7009953856468201)
('jerseys', 0.6800992488861084)
('colored', 0.6640898585319519)
('sash', 0.6634660959243774)


In [27]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brussels', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('maastricht', 0.7439447641372681)
('bruges', 0.7427038550376892)
('aachen', 0.7381912469863892)
('amsterdam', 0.737143337726593)
('ghent', 0.7288802266120911)
('strasbourg', 0.7250431180000305)
('frankfurt', 0.7092779874801636)
('rotterdam', 0.7078688144683838)
('bonn', 0.7050737142562866)
('bordeaux', 0.7041142582893372)


In [28]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('germany', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('austria', 0.8248653411865234)
('italy', 0.7685587406158447)
('belgium', 0.7650684714317322)
('switzerland', 0.7640849351882935)
('hungary', 0.7583723068237305)
('poland', 0.7563267946243286)
('netherlands', 0.7454272508621216)
('berlin', 0.7384282350540161)
('finland', 0.7346256375312805)
('denmark', 0.7233068943023682)


In [29]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('strange', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

('mysterious', 0.6952362656593323)
('fictionalised', 0.6614861488342285)
('kitten', 0.6589220762252808)
('scary', 0.6574631929397583)
('snowman', 0.6556798815727234)
('hideous', 0.6534916758537292)
('awful', 0.6533659100532532)
('hilarious', 0.6519839763641357)
('surreal', 0.6514548063278198)
('apparitions', 0.6504389047622681)


In [30]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['man'], positive=['king','woman'])
print('\n'.join([str(tup) for tup in analogous_words]))

('queen', 0.6321631669998169)
('throne', 0.6140472888946533)
('sigismund', 0.6035372018814087)
('daughter', 0.6032637357711792)
('olympias', 0.6022640466690063)
('prince', 0.6015709638595581)
('dowager', 0.59840989112854)
('marries', 0.5950711369514465)
('wenceslaus', 0.5947388410568237)
('isabella', 0.5926554203033447)


In [31]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['king'], positive=['queen','man'])
print('\n'.join([str(tup) for tup in analogous_words]))

('woman', 0.6597509980201721)
('frail', 0.5844504833221436)
('girl', 0.5768628120422363)
('blonde', 0.5707680583000183)
('prostitute', 0.5689617395401001)
('totoro', 0.5675539970397949)
('loving', 0.5577417612075806)
('gentleman', 0.552753210067749)
('ladies', 0.5510775446891785)
('pygmalion', 0.5509780645370483)


In [34]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

('son', 0.8452534675598145)
('brother', 0.7582974433898926)
('cousin', 0.7579344511032104)
('nephew', 0.7464486956596375)
('eldest', 0.743414044380188)
('wife', 0.7388955354690552)
('grandson', 0.7323840856552124)
('confidant', 0.7276127338409424)
('granddaughter', 0.7237993478775024)
('olympias', 0.7098314166069031)


In [36]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','girl'])
print('\n'.join([str(tup) for tup in analogous_words]))

('boy', 0.6801096200942993)
('bartender', 0.6520256996154785)
('man', 0.6491988897323608)
('boys', 0.6477783918380737)
('mister', 0.6474010348320007)
('ziegfeld', 0.6331601142883301)
('totoro', 0.6317894458770752)
('ugly', 0.6317366361618042)
('kid', 0.6297487616539001)
('naughty', 0.6279672980308533)


In [38]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

('daughters', 0.6616181135177612)
('sons', 0.593657910823822)
('eleonora', 0.5934041738510132)
('parents', 0.5883336663246155)
('mother', 0.5797499418258667)
('sister', 0.5784984827041626)
('sisters', 0.5775691270828247)
('alexandra', 0.5699713230133057)
('simeon', 0.5699524879455566)
('brothers', 0.5695198774337769)


## Using pre-trained models in Gensim

In many applications, you will simply want access to pre-trained word vectors (e.g. for plugging in to another model you are training). If you don't need the vectors to be tailored closely to your particular corpus, then you might like to use some pretrained models.

`word2vec` is not the only word embedding family of algorithms. Another, arguably even more powerful algorithm is the `FastText` algorithm, which Mikolov developed after moving to Facebook:

* Piotr Bojanowski and others, ‘Enriching Word Vectors with Subword Information’, ArXiv:1607.04606, 2017 <http://arxiv.org/abs/1607.04606>.

Instead of computing word vectors for each word, FastText splits each word into its constituent chunks. For example, 'cat' would be split into 'c', 'a', 't', 'ca', 'at' and 'cat', and 'burp' would be split into 'b', 'u', 'r', 'p', 'bu', 'ur', 'rp', 'bur', 'urp' and 'burp'. Then a vector is computer for each chunk that appears in the corpus. Each word is represented as the mean of all the chunks that make it up. FastText is able to learn very good word vectors because it can extract meaning from subword units, e.g. it can see that 'television', 'telegraph' and 'telepathy' all have 'tele' at the front, and can see that 'formality', 'criminality' and 'paucity' share subword units such as 'al' and 'ity'.

You can access many pretrained models using the Gensim downloader. Using the cells below, you can try out some of the different models available through Gensim. Along with `word2vec` and `FastText`, Gensim also supports `Glove` and `Doc2Vec` models.

**NB:** These trained models are very large, and will take a while to download. You may wish to download this notebook and execute the cells below on your own machine, in case Google kicks you out of the Colab environment.

In [39]:
# See what models are on offer
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# 300-dimensional word vectors trained on a huge dataset from Google News
google_news_w2v = api.load('word2vec-google-news-300')

In [ ]:
# x is to Kenya as Canberra is to Australia
google_news_w2v.similar_words(negative=['australia'],positive=['kenya','canberra'], topn=10)

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# Facebook's own FastText vectors, trained on Wikipedia
wikipedia_fasttext = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
# x is to Wharton as London is to Dickens
wikipedia_fasttext.similar_words(negative=['dickens'],positive=['wharton','london'], topn=10)

## Training an Armenian model

Now let's train a model on our own corpus. You can try to create Armenian / Ukrainian / Russian word vector model

Armenian, Ukrainian, Russian plaintext wikipedias are available at:
https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2735#

We can use the same size ~15 Million Words, (200MB) of these corpora to make sure the model is built within a reasonable time; however, when you have more time, you can try a more complete corpus.

It is possible to download/unzip/upload the files manually, however, it is faster to automatically load them directly from Heidelberg server space https://heibox.uni-heidelberg.de/d/42a07c9e95774e099a81/  into your colab environment. To do this run the following cells
